# "Playing Atari with Deep Reinforcement Learning" paper implementation - https://arxiv.org/pdf/1312.5602.pdf

In [3]:
import os
import re
import gymnasium as gym
import time
import copy
import random
import warnings
import numpy as np
import torch
import torchvision
import torch.nn as nn
from IPython import display
from skimage.color import rgb2gray
from skimage.transform import rescale
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
from collections import deque, namedtuple

plt.style.use('seaborn')
warnings.filterwarnings('ignore')

/tmp/ipykernel_1964/384864923.py:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [4]:
import torch.nn as nn

class DeepQNetwork(nn.Module):
    def __init__(self, num_frames, num_actions):
        super(DeepQNetwork, self).__init__()
        self.num_frames = num_frames
        self.num_actions = num_actions

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=num_frames, out_channels=16, kernel_size=8, stride=4, padding=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=4, stride=2, padding=1),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=3200, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=num_actions)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
